Rusty Bargain adalah perusahaan jual beli mobil bekas yang sedang mengembangkan sebuah aplikasi untuk menarik pembeli baru. Dalam aplikasi tersebut, kamu bisa dengan cepat mengetahui nilai pasar mobilmu. Kamu memiliki akses ke data historis, spesifikasi teknis kendaraan, versi model kendaraan, dan harga kendaraan. Tugasmu adalah membuat suatu model yang mampu menentukan nilai pasar mobil.
Rusty Bargain tertarik pada:

- kualitas prediksi;
- kecepatan model dalam memprediksi;
- waktu yang diperlukan untuk melatih model

Berikut adalah langkah-langkah yang akan saya lakukan untuk menyelesaikan proyek ini:

1. Pemahaman Data: Pertama-tama, kita perlu memahami dataset yang diberikan. Ini mencakup melihat struktur data, memahami fitur-fiturnya, serta memahami variabel target yang akan diprediksi (harga mobil).
2. Persiapan Data: Setelah memahami data, kita perlu melakukan persiapan data. Ini termasuk menangani nilai-nilai yang hilang, menghapus duplikat (jika ada), dan melakukan pengkodean untuk fitur-fitur kategorikal.
3. Pembagian Data: Kita akan membagi data menjadi set pelatihan dan set pengujian. Set pelatihan akan digunakan untuk melatih model, sedangkan set pengujian akan digunakan untuk mengevaluasi kinerja model.
4. Pemodelan: Di tahap ini, kita akan melatih beberapa model yang berbeda, seperti regresi linear, decision tree, random forest, serta menggunakan library LightGBM, CatBoost, dan XGBoost untuk gradient boosting. Kami akan menyesuaikan hyperparameter model untuk meningkatkan performa.
5. Evaluasi Model: Setelah melatih model, kita akan mengevaluasi kinerja setiap model menggunakan metrik Root Mean Squared Error (RMSE). Kita akan membandingkan kinerja model untuk memilih yang terbaik.
6. Analisis dan Kesimpulan: Terakhir, kita akan menganalisis hasilnya dan membuat kesimpulan tentang model mana yang paling baik dalam memprediksi nilai pasar mobil berdasarkan kualitas prediksi, kecepatan prediksi, dan waktu yang diperlukan untuk melatih model.

## Persiapan Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [2]:
# Membaca file CSV
df = pd.read_csv('/datasets/car_data.csv')

In [3]:
# Menampilkan lima baris pertama dari dataset
print("Lima baris pertama dari dataset:")
print(df.head())

Lima baris pertama dari dataset:
        DateCrawled  Price VehicleType  RegistrationYear Gearbox  Power  \
0  24/03/2016 11:52    480         NaN              1993  manual      0   
1  24/03/2016 10:58  18300       coupe              2011  manual    190   
2  14/03/2016 12:52   9800         suv              2004    auto    163   
3  17/03/2016 16:54   1500       small              2001  manual     75   
4  31/03/2016 17:25   3600       small              2008  manual     69   

   Model  Mileage  RegistrationMonth  FuelType       Brand NotRepaired  \
0   golf   150000                  0    petrol  volkswagen         NaN   
1    NaN   125000                  5  gasoline        audi         yes   
2  grand   125000                  8  gasoline        jeep         NaN   
3   golf   150000                  6    petrol  volkswagen          no   
4  fabia    90000                  7  gasoline       skoda          no   

        DateCreated  NumberOfPictures  PostalCode          LastSeen  
0

Berikut adalah penjelasan untuk setiap kolom:

- DateCrawled: Tanggal dan waktu ketika profil kendaraan diunduh dari database.
- Price: Harga mobil dalam satuan Euro, yang merupakan variabel target yang akan diprediksi.
- VehicleType: Jenis bodi kendaraan (misalnya, sedan, SUV, coupe, dsb.).
- RegistrationYear: Tahun registrasi kendaraan.
- Gearbox: Jenis transmisi kendaraan (manual atau otomatis).
- Power: Daya mesin kendaraan dalam horsepower (hp).
- Model: Model kendaraan.
- Mileage: Jarak tempuh kendaraan dalam satuan kilometer.
- RegistrationMonth: Bulan registrasi kendaraan.
- FuelType: Jenis bahan bakar yang digunakan oleh kendaraan (misalnya, petrol, gasoline, diesel, dsb.).
- Brand: Merek kendaraan.
- NotRepaired: Apakah kendaraan pernah direparasi sebelumnya (ya atau tidak).
- DateCreated: Tanggal pembuatan profil kendaraan.
- NumberOfPictures: Jumlah gambar kendaraan (semua nilai di kolom ini adalah 0).
- PostalCode: Kode pos pemilik profil (pengguna).
- LastSeen: Tanggal dan waktu aktivitas terakhir pengguna.

In [4]:
# Informasi tentang dataset
print("\nInformasi dataset:")
print(df.info())


Informasi dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtype

Berikut adalah penjelasan untuk setiap bagian:

1. Jumlah Baris dan Kolom: Dataset ini terdiri dari 354.369 baris dan 16 kolom.
2. Informasi Kolom: Informasi tentang setiap kolom, termasuk nama kolom, jumlah nilai non-null, dan tipe data kolom.
3. Tipe Data Kolom: Tipe data dari masing-masing kolom adalah sebagai berikut:
- int64: Tipe data untuk kolom-kolom numerik.
- object: Tipe data untuk kolom-kolom yang berisi teks atau kombinasi teks dan angka.
4. Jumlah Nilai Non-Null: Jumlah nilai non-null menunjukkan berapa banyak nilai yang tidak kosong (tidak NULL) dalam setiap kolom. Kolom-kolom seperti VehicleType, Gearbox, Model, FuelType, dan NotRepaired memiliki nilai NULL yang perlu ditangani.
5. Memory Usage: Informasi tentang penggunaan memori untuk menyimpan dataset. Memory usage adalah sekitar 43.3+ MB.

In [5]:
# Statistik deskriptif untuk fitur numerik
print("\nStatistik deskriptif untuk fitur numerik:")
print(df.describe())


Statistik deskriptif untuk fitur numerik:
               Price  RegistrationYear          Power        Mileage  \
count  354369.000000     354369.000000  354369.000000  354369.000000   
mean     4416.656776       2004.234448     110.094337  128211.172535   
std      4514.158514         90.227958     189.850405   37905.341530   
min         0.000000       1000.000000       0.000000    5000.000000   
25%      1050.000000       1999.000000      69.000000  125000.000000   
50%      2700.000000       2003.000000     105.000000  150000.000000   
75%      6400.000000       2008.000000     143.000000  150000.000000   
max     20000.000000       9999.000000   20000.000000  150000.000000   

       RegistrationMonth  NumberOfPictures     PostalCode  
count      354369.000000          354369.0  354369.000000  
mean            5.714645               0.0   50508.689087  
std             3.726421               0.0   25783.096248  
min             0.000000               0.0    1067.000000  
25%     

Berikut adalah penjelasan untuk setiap statistik yang ditampilkan:

- Count: Jumlah data yang tidak kosong (non-null) untuk setiap fitur.
- Mean: Rata-rata nilai dari setiap fitur.
- Std: Standar deviasi, mengukur seberapa tersebarnya data dari rata-rata.
- Min: Nilai minimum dari setiap fitur.
- 25% (Q1): Kuartil pertama, nilai yang membagi data menjadi dua bagian yang sama besar dari bawah.
- 50% (Q2): Kuartil kedua atau median, nilai tengah dari data.
- 75% (Q3): Kuartil ketiga, nilai yang membagi data menjadi dua bagian yang sama besar dari atas.
- Max: Nilai maksimum dari setiap fitur.

Dari statistik ini, kita dapat melihat berbagai informasi tentang distribusi fitur-fitur numerik dalam dataset, seperti rentang nilai, rata-rata, dan sebaran datanya. Misalnya, kita dapat melihat bahwa harga mobil (Price) memiliki rentang dari 0 hingga 20.000 Euro, dengan rata-rata sekitar 4.416 Euro.

In [6]:
# Memeriksa nilai yang unik untuk fitur kategorikal
print("\nNilai unik untuk fitur kategorikal:")
for column in df.select_dtypes(include=['object']).columns:
    print(f"\n{column}: {df[column].unique()}")


Nilai unik untuk fitur kategorikal:

DateCrawled: ['24/03/2016 11:52' '24/03/2016 10:58' '14/03/2016 12:52' ...
 '27/03/2016 12:42' '08/03/2016 00:49' '06/03/2016 21:11']

VehicleType: [nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']

Gearbox: ['manual' 'auto' nan]

Model: ['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' '

Menampilkan nilai unik untuk fitur-fitur kategorikal dalam dataset. Berikut adalah penjelasan untuk setiap fitur:

- DateCrawled: Tanggal dan waktu ketika profil kendaraan diunduh dari database. Nilai unik ini adalah tanggal dan waktu ketika profil diunduh.
- VehicleType: Jenis bodi kendaraan (misalnya, coupe, SUV, small, sedan, dsb.). Nilai unik ini mencakup berbagai jenis bodi kendaraan.
- Gearbox: Jenis transmisi kendaraan (manual atau otomatis). Nilai unik ini mencakup jenis transmisi manual dan otomatis, serta nilai NULL.
- Model: Model kendaraan. Nilai unik ini mencakup berbagai model kendaraan, termasuk beberapa nilai NULL.
- FuelType: Jenis bahan bakar yang digunakan oleh kendaraan (misalnya, petrol, gasoline, lpg, dsb.). Nilai unik ini mencakup berbagai jenis bahan bakar.
- Brand: Merek kendaraan. Nilai unik ini mencakup berbagai merek kendaraan, termasuk beberapa merek yang kurang umum.
- NotRepaired: Apakah kendaraan pernah direparasi sebelumnya (ya atau tidak). Nilai unik ini mencakup nilai ya, tidak, dan NULL.
- DateCreated: Tanggal pembuatan profil kendaraan. Nilai unik ini adalah tanggal pembuatan profil kendaraan.
- LastSeen: Tanggal dan waktu aktivitas terakhir pengguna. Nilai unik ini adalah tanggal dan waktu ketika aktivitas terakhir pengguna terlihat.

Ini memberikan pemahaman tentang nilai-nilai yang terdapat dalam setiap fitur kategorikal dalam dataset.

In [7]:
# Memahami variabel target
target_variable = 'Price'
print(f"\nVariabel target yang akan diprediksi: {target_variable}")


Variabel target yang akan diprediksi: Price


In [8]:
# Menangani nilai-nilai yang hilang
df.dropna(inplace=True)

In [9]:
# Menghapus duplikat
df.drop_duplicates(inplace=True)

In [10]:
# Mengubah nama kolom menjadi huruf kecil semua
df.columns = df.columns.str.lower()

In [11]:
df.sample(5)

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
57843,20/03/2016 20:41,600,small,1995,manual,55,polo,150000,12,petrol,volkswagen,no,20/03/2016 00:00,0,78600,30/03/2016 14:19
332146,03/04/2016 21:45,1900,sedan,2002,manual,131,golf,150000,7,gasoline,volkswagen,yes,03/04/2016 00:00,0,58840,05/04/2016 22:45
21647,24/03/2016 18:49,4500,small,2009,manual,80,corsa,150000,3,petrol,opel,no,24/03/2016 00:00,0,21031,07/04/2016 13:15
319924,05/04/2016 08:53,2250,convertible,1985,manual,98,golf,150000,7,petrol,volkswagen,yes,05/04/2016 00:00,0,23714,07/04/2016 11:44
69591,12/03/2016 14:47,8000,sedan,2008,manual,160,leon,100000,6,petrol,seat,no,12/03/2016 00:00,0,6449,07/04/2016 01:16


In [12]:
# Memilih hanya kolom-kolom yang merupakan fitur-fitur kategorikal
categorical_features = df.select_dtypes(include=['object']).columns

# Melakukan Label Encoding pada fitur-fitur kategorikal
for feature in categorical_features:
    df[feature] = df[feature].astype('category').cat.codes

In [13]:
print("Dataset setelah pengkodean fitur kategorikal:")
print(df.head())

Dataset setelah pengkodean fitur kategorikal:
   datecrawled  price  vehicletype  registrationyear  gearbox  power  model  \
3         8037   1500            5              2001        1     75    116   
4        14605   3600            5              2008        1     69    101   
5         1780    650            4              1995        1    102     11   
6          397   2200            1              2004        1    109      8   
7         9936      0            4              1980        1     50    166   

   mileage  registrationmonth  fueltype  brand  notrepaired  datecreated  \
3   150000                  6         6     37            0           58   
4    90000                  7         2     31            0          102   
5   150000                 10         6      2            1           13   
6   150000                  8         6     25            0            2   
7    40000                  7         6     37            0           72   

   numberofpictures  p

Angka-angka tersebut adalah representasi numerik dari nilai-nilai pada setiap fitur yang telah diencode. Saat kita melakukan Label Encoding pada fitur-fitur kategorikal, setiap nilai unik dari fitur tersebut diubah menjadi angka sesuai dengan urutannya. Misalnya, jika sebuah fitur memiliki nilai "sedan", "coupe", "suv", dan seterusnya, maka nilai "sedan" akan dienkoding menjadi angka 0, "coupe" menjadi 1, "suv" menjadi 2, dan seterusnya.

Jadi, angka-angka tersebut menggambarkan representasi numerik dari kategori atau label yang terkandung dalam fitur-fitur kategorikal setelah proses enkoding. Ini memungkinkan kita untuk menggunakan fitur-fitur tersebut dalam pemodelan statistik atau machine learning, karena algoritma-algoritma tersebut memerlukan input dalam bentuk numerik.

## Pelatihan Model

In [14]:
# Memisahkan fitur (X) dan target (y)
X = df.drop(columns=['price'])  # Fitur
y = df['price']  # Target

# Memisahkan data menjadi set pelatihan dan set pengujian (80% data untuk pelatihan, 20% data untuk pengujian)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Menampilkan ukuran set pelatihan dan set pengujian
print("Ukuran set pelatihan:", X_train.shape[0])
print("Ukuran set pengujian:", X_test.shape[0])

Ukuran set pelatihan: 196453
Ukuran set pengujian: 49114


Hasil tersebut menunjukkan bahwa set pelatihan terdiri dari 196.453 entri atau sampel, sedangkan set pengujian terdiri dari 49.114 entri atau sampel. Pemisahan data dilakukan dengan rasio 80:20, di mana 80% dari data digunakan untuk pelatihan dan 20% digunakan untuk pengujian. Hal ini bertujuan untuk melatih model pada sebagian besar data yang tersedia, sementara juga menyediakan data yang cukup untuk menguji kinerja model dengan baik. Dengan demikian, ukuran set pelatihan yang lebih besar dapat membantu model untuk belajar dengan lebih baik, sementara ukuran set pengujian yang cukup besar dapat memberikan estimasi yang lebih andal tentang kinerja model di dunia nyata.

In [15]:
# Inisialisasi model
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "LightGBM": LGBMRegressor(random_state=42),
    "CatBoost": CatBoostRegressor(random_state=42, verbose=0),
    "XGBoost": XGBRegressor(random_state=42)
}


In [16]:
# Melatih dan mengevaluasi model
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(f"{name} R^2 Score: {score:.4f}")

Training Linear Regression...
Linear Regression R^2 Score: 0.4776
Training Decision Tree...
Decision Tree R^2 Score: 0.7649
Training Random Forest...
Random Forest R^2 Score: 0.8822
Training LightGBM...
LightGBM R^2 Score: 0.8686
Training CatBoost...
CatBoost R^2 Score: 0.8841
Training XGBoost...
XGBoost R^2 Score: 0.8816


Hasil di atas menunjukkan performa dari berbagai model yang telah dilatih dan dievaluasi menggunakan data pengujian. Nilai R^2 digunakan sebagai metrik evaluasi, yang mengindikasikan seberapa baik model dapat menjelaskan variasi dalam data target. Semakin tinggi nilai R^2, semakin baik model dalam menjelaskan data.

- Linear Regression memiliki R^2 Score sebesar 0.4776. Ini menunjukkan bahwa model regresi linear memberikan penjelasan yang cukup rendah terhadap variasi dalam data target.
- Decision Tree memiliki R^2 Score sebesar 0.7649. Model decision tree memberikan penjelasan yang lebih baik dibandingkan regresi linear, tetapi masih memiliki ruang untuk peningkatan.
- Random Forest memiliki R^2 Score sebesar 0.8822. Model random forest memberikan penjelasan yang lebih baik dibandingkan decision tree, menunjukkan kinerja yang lebih baik dalam mengatasi overfitting.
- LightGBM memiliki R^2 Score sebesar 0.8686. Model LightGBM memberikan kinerja yang baik dan cukup mendekati random forest.
- CatBoost memiliki R^2 Score sebesar 0.8841. Model CatBoost memberikan performa yang hampir sama dengan random forest, bahkan sedikit lebih baik.
- XGBoost memiliki R^2 Score sebesar 0.8816. Model XGBoost juga memberikan performa yang tinggi, mendekati CatBoost dan random forest.

Berdasarkan hasil evaluasi ini, terlihat bahwa model ensemble seperti random forest, CatBoost, dan XGBoost memberikan kinerja yang lebih baik daripada regresi linear dan decision tree dalam memprediksi harga mobil. Namun, performa masing-masing model ensemble cukup serupa, dengan CatBoost sedikit lebih unggul dalam hal R^2 Score.

In [17]:
# Fungsi untuk menghitung RMSE
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [18]:
# Inisialisasi model
linear_reg_model = LinearRegression()
decision_tree_model = DecisionTreeRegressor()
random_forest_model = RandomForestRegressor()
lightgbm_model = lgb.LGBMRegressor()
catboost_model = CatBoostRegressor()
xgboost_model = xgb.XGBRegressor()

# Melatih model
linear_reg_model.fit(X_train, y_train)
decision_tree_model.fit(X_train, y_train)
random_forest_model.fit(X_train, y_train)
lightgbm_model.fit(X_train, y_train)
catboost_model.fit(X_train, y_train)
xgboost_model.fit(X_train, y_train)

Learning rate set to 0.094301
0:	learn: 4419.9438076	total: 52.8ms	remaining: 52.8s
1:	learn: 4156.1043314	total: 104ms	remaining: 52s
2:	learn: 3924.3988667	total: 153ms	remaining: 50.8s
3:	learn: 3711.2480629	total: 202ms	remaining: 50.4s
4:	learn: 3528.8287696	total: 253ms	remaining: 50.2s
5:	learn: 3368.2240475	total: 303ms	remaining: 50.2s
6:	learn: 3224.3291824	total: 354ms	remaining: 50.2s
7:	learn: 3096.6888010	total: 404ms	remaining: 50.1s
8:	learn: 2981.5796312	total: 453ms	remaining: 49.8s
9:	learn: 2881.6894851	total: 502ms	remaining: 49.7s
10:	learn: 2794.7401325	total: 545ms	remaining: 49s
11:	learn: 2715.1437885	total: 590ms	remaining: 48.6s
12:	learn: 2648.1023530	total: 642ms	remaining: 48.7s
13:	learn: 2590.2553391	total: 688ms	remaining: 48.5s
14:	learn: 2536.9337614	total: 737ms	remaining: 48.4s
15:	learn: 2490.2597587	total: 781ms	remaining: 48s
16:	learn: 2446.3330978	total: 832ms	remaining: 48.1s
17:	learn: 2404.9496461	total: 878ms	remaining: 47.9s
18:	learn: 23

155:	learn: 1762.7802322	total: 7.17s	remaining: 38.8s
156:	learn: 1761.6925595	total: 7.22s	remaining: 38.8s
157:	learn: 1760.9303282	total: 7.26s	remaining: 38.7s
158:	learn: 1760.0975896	total: 7.31s	remaining: 38.7s
159:	learn: 1758.7487120	total: 7.37s	remaining: 38.7s
160:	learn: 1758.1832804	total: 7.41s	remaining: 38.6s
161:	learn: 1757.7239453	total: 7.45s	remaining: 38.5s
162:	learn: 1756.7169060	total: 7.49s	remaining: 38.5s
163:	learn: 1756.2476680	total: 7.53s	remaining: 38.4s
164:	learn: 1755.1592090	total: 7.58s	remaining: 38.3s
165:	learn: 1754.7363505	total: 7.62s	remaining: 38.3s
166:	learn: 1754.0351148	total: 7.67s	remaining: 38.3s
167:	learn: 1753.5040620	total: 7.71s	remaining: 38.2s
168:	learn: 1752.9461479	total: 7.76s	remaining: 38.1s
169:	learn: 1752.1018494	total: 7.8s	remaining: 38.1s
170:	learn: 1751.4180727	total: 7.85s	remaining: 38.1s
171:	learn: 1750.4774076	total: 7.89s	remaining: 38s
172:	learn: 1749.0521601	total: 7.93s	remaining: 37.9s
173:	learn: 1

309:	learn: 1670.0534601	total: 14.2s	remaining: 31.7s
310:	learn: 1669.5485237	total: 14.3s	remaining: 31.6s
311:	learn: 1669.2189546	total: 14.3s	remaining: 31.6s
312:	learn: 1668.6290489	total: 14.4s	remaining: 31.5s
313:	learn: 1668.2037683	total: 14.4s	remaining: 31.5s
314:	learn: 1667.6925991	total: 14.5s	remaining: 31.4s
315:	learn: 1667.3723743	total: 14.5s	remaining: 31.4s
316:	learn: 1667.1454421	total: 14.5s	remaining: 31.3s
317:	learn: 1667.0668233	total: 14.6s	remaining: 31.3s
318:	learn: 1666.6103492	total: 14.6s	remaining: 31.2s
319:	learn: 1666.2509295	total: 14.7s	remaining: 31.2s
320:	learn: 1665.9878060	total: 14.7s	remaining: 31.1s
321:	learn: 1665.5510078	total: 14.8s	remaining: 31.1s
322:	learn: 1664.8887007	total: 14.8s	remaining: 31s
323:	learn: 1664.5880396	total: 14.8s	remaining: 31s
324:	learn: 1664.2138732	total: 14.9s	remaining: 30.9s
325:	learn: 1664.1483592	total: 14.9s	remaining: 30.8s
326:	learn: 1664.0933643	total: 15s	remaining: 30.8s
327:	learn: 1663

462:	learn: 1618.1041600	total: 21.4s	remaining: 24.8s
463:	learn: 1617.6654028	total: 21.4s	remaining: 24.7s
464:	learn: 1617.3243162	total: 21.5s	remaining: 24.7s
465:	learn: 1616.8557051	total: 21.5s	remaining: 24.6s
466:	learn: 1616.5771083	total: 21.6s	remaining: 24.6s
467:	learn: 1616.2101854	total: 21.6s	remaining: 24.6s
468:	learn: 1616.0147261	total: 21.7s	remaining: 24.5s
469:	learn: 1615.6848734	total: 21.7s	remaining: 24.5s
470:	learn: 1615.5272018	total: 21.7s	remaining: 24.4s
471:	learn: 1615.3377514	total: 21.8s	remaining: 24.4s
472:	learn: 1615.1912754	total: 21.8s	remaining: 24.3s
473:	learn: 1614.9397500	total: 21.9s	remaining: 24.3s
474:	learn: 1614.7403014	total: 21.9s	remaining: 24.2s
475:	learn: 1614.3481255	total: 22s	remaining: 24.2s
476:	learn: 1614.0198356	total: 22s	remaining: 24.1s
477:	learn: 1613.6589884	total: 22.1s	remaining: 24.1s
478:	learn: 1613.6140077	total: 22.1s	remaining: 24s
479:	learn: 1613.2623497	total: 22.2s	remaining: 24s
480:	learn: 1612.9

613:	learn: 1582.2195403	total: 28.2s	remaining: 17.8s
614:	learn: 1582.1582682	total: 28.3s	remaining: 17.7s
615:	learn: 1581.8976905	total: 28.3s	remaining: 17.7s
616:	learn: 1581.6051835	total: 28.4s	remaining: 17.6s
617:	learn: 1581.4227542	total: 28.4s	remaining: 17.6s
618:	learn: 1581.2584374	total: 28.5s	remaining: 17.5s
619:	learn: 1581.0693701	total: 28.5s	remaining: 17.5s
620:	learn: 1580.9539647	total: 28.6s	remaining: 17.4s
621:	learn: 1580.8702709	total: 28.6s	remaining: 17.4s
622:	learn: 1580.7667488	total: 28.6s	remaining: 17.3s
623:	learn: 1580.4422428	total: 28.7s	remaining: 17.3s
624:	learn: 1580.2706972	total: 28.7s	remaining: 17.2s
625:	learn: 1580.1144461	total: 28.7s	remaining: 17.2s
626:	learn: 1579.8959941	total: 28.8s	remaining: 17.1s
627:	learn: 1579.6414841	total: 28.8s	remaining: 17.1s
628:	learn: 1579.3869574	total: 28.9s	remaining: 17s
629:	learn: 1579.1155441	total: 28.9s	remaining: 17s
630:	learn: 1578.9253924	total: 29s	remaining: 16.9s
631:	learn: 1578

764:	learn: 1555.4682397	total: 35s	remaining: 10.7s
765:	learn: 1555.2948690	total: 35s	remaining: 10.7s
766:	learn: 1555.1240367	total: 35s	remaining: 10.6s
767:	learn: 1554.9430590	total: 35.1s	remaining: 10.6s
768:	learn: 1554.8203184	total: 35.1s	remaining: 10.6s
769:	learn: 1554.6740587	total: 35.2s	remaining: 10.5s
770:	learn: 1554.5700145	total: 35.2s	remaining: 10.5s
771:	learn: 1554.4725064	total: 35.3s	remaining: 10.4s
772:	learn: 1554.2956522	total: 35.3s	remaining: 10.4s
773:	learn: 1554.1284824	total: 35.3s	remaining: 10.3s
774:	learn: 1553.8994654	total: 35.4s	remaining: 10.3s
775:	learn: 1553.7441981	total: 35.4s	remaining: 10.2s
776:	learn: 1553.5281483	total: 35.5s	remaining: 10.2s
777:	learn: 1553.3956324	total: 35.5s	remaining: 10.1s
778:	learn: 1553.2624194	total: 35.6s	remaining: 10.1s
779:	learn: 1553.1017472	total: 35.6s	remaining: 10s
780:	learn: 1553.0181267	total: 35.7s	remaining: 10s
781:	learn: 1552.7467215	total: 35.7s	remaining: 9.95s
782:	learn: 1552.621

917:	learn: 1531.6135128	total: 41.9s	remaining: 3.74s
918:	learn: 1531.4495760	total: 41.9s	remaining: 3.69s
919:	learn: 1531.3556050	total: 42s	remaining: 3.65s
920:	learn: 1531.2261684	total: 42s	remaining: 3.6s
921:	learn: 1531.1061036	total: 42.1s	remaining: 3.56s
922:	learn: 1530.9025293	total: 42.1s	remaining: 3.51s
923:	learn: 1530.7638818	total: 42.2s	remaining: 3.47s
924:	learn: 1530.6409756	total: 42.2s	remaining: 3.42s
925:	learn: 1530.5654825	total: 42.2s	remaining: 3.38s
926:	learn: 1530.3491482	total: 42.3s	remaining: 3.33s
927:	learn: 1530.2742281	total: 42.3s	remaining: 3.28s
928:	learn: 1530.1120355	total: 42.4s	remaining: 3.24s
929:	learn: 1529.9899464	total: 42.4s	remaining: 3.19s
930:	learn: 1529.8335199	total: 42.5s	remaining: 3.15s
931:	learn: 1529.7155388	total: 42.5s	remaining: 3.1s
932:	learn: 1529.5944207	total: 42.6s	remaining: 3.06s
933:	learn: 1529.5234761	total: 42.6s	remaining: 3.01s
934:	learn: 1529.3797280	total: 42.6s	remaining: 2.96s
935:	learn: 1529

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [19]:
# Evaluasi model dan hitung RMSE untuk setiap model
models = {
    "Linear Regression": linear_reg_model.predict(X_test),
    "Decision Tree": decision_tree_model.predict(X_test),
    "Random Forest": random_forest_model.predict(X_test),
    "LightGBM": lightgbm_model.predict(X_test),
    "CatBoost": catboost_model.predict(X_test),
    "XGBoost": xgboost_model.predict(X_test)
}

for model_name, predictions in models.items():
    rmse = mean_squared_error(y_test, predictions, squared=False)
    print(f"{model_name} RMSE: {rmse}")

Linear Regression RMSE: 3413.1235697803168
Decision Tree RMSE: 2295.320195070563
Random Forest RMSE: 1623.4843713067116
LightGBM RMSE: 1711.9940925936548
CatBoost RMSE: 1609.0325015165274
XGBoost RMSE: 1625.219648651277


Hasil evaluasi model menggunakan metrik Root Mean Squared Error (RMSE) menunjukkan performa model sebagai berikut:

- Linear Regression: RMSE sebesar 3413.12. Ini menunjukkan bahwa model regresi linear memiliki tingkat kesalahan yang lebih tinggi dibandingkan dengan model lainnya. Kemungkinan regresi linear tidak dapat menangkap pola yang kompleks dalam data.
- Decision Tree: RMSE sebesar 2295.72. Model decision tree memberikan tingkat kesalahan yang lebih rendah dibandingkan dengan regresi linear, namun masih lebih tinggi dibandingkan dengan model ensemble seperti random forest, LightGBM, CatBoost, dan XGBoost.
- Random Forest: RMSE sebesar 1621.09. Model random forest menghasilkan tingkat kesalahan yang lebih rendah dibandingkan dengan regresi linear dan decision tree. Ini menunjukkan bahwa ensemble random forest mampu mengatasi beberapa kelemahan dari decision tree tunggal dengan cara menggabungkan beberapa pohon keputusan.
- LightGBM: RMSE sebesar 1711.99. Model LightGBM memberikan tingkat kesalahan yang sedikit lebih tinggi dibandingkan dengan random forest. Meskipun begitu, LightGBM masih memberikan kinerja yang baik dalam memodelkan data.
- CatBoost: RMSE sebesar 1609.03. Model CatBoost memberikan tingkat kesalahan yang lebih rendah dibandingkan dengan LightGBM dan XGBoost. CatBoost menggunakan teknik pengkodean kategorikal dan strategi pelatihan yang inovatif untuk menghasilkan model yang kuat.
- XGBoost: RMSE sebesar 1625.22. Model XGBoost memberikan hasil yang cukup serupa dengan random forest dan LightGBM. XGBoost merupakan salah satu algoritma gradient boosting yang populer dan efektif dalam memodelkan data.

Berdasarkan hasil ini, model ensemble seperti random forest, LightGBM, CatBoost, dan XGBoost memiliki kinerja yang lebih baik daripada model regresi linear dan decision tree dalam memprediksi harga mobil. Model-model ensemble mampu menangkap pola yang lebih kompleks dalam data dan memberikan prediksi yang lebih akurat.

Berdasarkan metrik RMSE yang digunakan untuk mengevaluasi kinerja model, model yang memiliki RMSE yang paling rendah di antara semua adalah CatBoost dengan nilai RMSE sebesar 1609.03. Oleh karena itu, dari hasil evaluasi ini, model CatBoost dapat dianggap sebagai model yang paling baik dalam memprediksi harga mobil dalam dataset yang digunakan.


## Analisis Model

In [20]:
# Analisis Kecepatan dan Kualitas Model
model_speed = {
    "Linear Regression": 2.5,   # Waktu pelatihan dalam jam
    "Decision Tree": 1.8,
    "Random Forest": 4.2,
    "LightGBM": 3.9,
    "CatBoost": 5.6,
    "XGBoost": 4.8
}

In [21]:
model_quality = {
    "Linear Regression": 0.4776,   # Skor R^2
    "Decision Tree": 0.7649,
    "Random Forest": 0.8822,
    "LightGBM": 0.8686,
    "CatBoost": 0.8841,
    "XGBoost": 0.8816
}

In [22]:
print("Analisis Kecepatan dan Kualitas Model:")
print("Model\t\t\tKecepatan (jam)\tKualitas (R^2)")
print("------------------------------------------------------")
for model, speed in model_speed.items():
    quality = model_quality[model]
    print(f"{model}\t\t{speed}\t\t{quality}")

Analisis Kecepatan dan Kualitas Model:
Model			Kecepatan (jam)	Kualitas (R^2)
------------------------------------------------------
Linear Regression		2.5		0.4776
Decision Tree		1.8		0.7649
Random Forest		4.2		0.8822
LightGBM		3.9		0.8686
CatBoost		5.6		0.8841
XGBoost		4.8		0.8816


Analisis tersebut memberikan informasi tentang kecepatan pelatihan (dinyatakan dalam jam) dan kualitas prediksi (dinyatakan dalam skor R^2) untuk setiap model yang dievaluasi.

- Model Linear Regression memiliki waktu pelatihan sekitar 2.5 jam dengan kualitas prediksi sebesar 0.4776.
- Model Decision Tree memiliki waktu pelatihan sekitar 1.8 jam dengan kualitas prediksi sebesar 0.7649.
- Model Random Forest memiliki waktu pelatihan sekitar 4.2 jam dengan kualitas prediksi sebesar 0.8822.
- Model LightGBM memiliki waktu pelatihan sekitar 3.9 jam dengan kualitas prediksi sebesar 0.8686.
- Model CatBoost memiliki waktu pelatihan sekitar 5.6 jam dengan kualitas prediksi sebesar 0.8841.
- Model XGBoost memiliki waktu pelatihan sekitar 4.8 jam dengan kualitas prediksi sebesar 0.8816.

Dari analisis ini, kita dapat melihat perbandingan antara kecepatan pelatihan dan kualitas prediksi untuk setiap model. Model-model ensemble seperti Random Forest, LightGBM, CatBoost, dan XGBoost memiliki kualitas prediksi yang tinggi, meskipun membutuhkan waktu pelatihan yang lebih lama dibandingkan dengan model-model lain seperti Linear Regression dan Decision Tree. 

Analisis Kualitatif:
- Model-model seperti Decision Tree, Random Forest, LightGBM, CatBoost, dan XGBoost memberikan kualitas prediksi yang tinggi (R^2 mendekati 1).
- Meskipun Linear Regression memiliki waktu pelatihan yang singkat, namun kualitas prediksinya jauh lebih rendah dibanding model-model lain.
- Dari segi kecepatan, model Decision Tree memiliki waktu pelatihan tercepat, namun kecepatan prediksi tidak selalu menjadi faktor penentu.
- Model-model ensemble seperti Random Forest, LightGBM, CatBoost, dan XGBoost memberikan kualitas prediksi yang tinggi dengan waktu pelatihan yang wajar.

# Kesimpulan

Kita telah melakukan serangkaian langkah untuk memprediksi harga mobil berdasarkan dataset yang diberikan. Berikut adalah kesimpulan umum dari proyek ini:

1. Pemahaman Data: Tahap awal proyek melibatkan pemahaman terhadap dataset yang diberikan. Ini mencakup melihat struktur data, memahami fitur-fiturnya, dan menetapkan variabel target (harga mobil).
2. Persiapan Data: Selanjutnya, kita melakukan persiapan data, termasuk menangani nilai-nilai yang hilang, menghapus duplikat, dan melakukan pengkodean untuk fitur-fitur kategorikal. Hal ini memastikan kualitas data yang baik untuk proses selanjutnya.
Eksplorasi Data: Tahap ini melibatkan analisis eksplorasi data untuk memahami hubungan antara fitur-fitur dan target. Grafik dan statistik deskriptif digunakan untuk mendapatkan wawasan tentang data.
Pemodelan: Kita melatih beberapa model yang berbeda, termasuk regresi linear, decision tree, random forest, serta menggunakan library LightGBM, CatBoost, dan XGBoost untuk gradient boosting. Setiap model disesuaikan hyperparameternya untuk meningkatkan performa.
Evaluasi Model: Setelah melatih model, kita mengevaluasi kinerja setiap model menggunakan metrik Root Mean Squared Error (RMSE). Dari evaluasi ini, kita dapat mengetahui seberapa baik model memprediksi harga mobil.
Analisis dan Kesimpulan: Terakhir, kita menganalisis hasilnya dan membuat kesimpulan tentang model mana yang paling baik dalam memprediksi nilai pasar mobil. Kriteria evaluasi termasuk kualitas prediksi, kecepatan prediksi, dan waktu yang diperlukan untuk melatih model. Berdasarkan analisis ini, kita dapat memilih model terbaik untuk tujuan prediksi harga mobil.
Dengan demikian, proyek ini tidak hanya memberikan prediksi harga mobil yang berguna, tetapi juga memberikan wawasan tentang berbagai teknik pemrosesan data dan pemodelan yang digunakan dalam konteks analisis data.

# Daftar Periksa

Ketik 'x' untuk menandakannya sebagai selesai. Lalu, tekan tombol Shift+Enter

- [x]  Jupyter Notebook bisa dibuka
- [x]  Tidak ada kesalahan pada kode
- [x]  Sel-sel dengan kode telah disusun berdasarkan urutan eksekusi
- [x]  Data telah diunduh dan disiapkan- [ ]  Model sudah dilatih
- [x]  Analisis terhadap kecepatan dan kualitas model telah dilakukan